<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Outlier_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load and controll cleaned data

In [3]:
# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/contracts_with_new_data_MEAN.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())

<ipython-input-3-283dbef524b6>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


   Unnamed: 0    ANO_SID CORPORATE_DEVISION  ORTPLZ  ORTS-NAME  \
0           0  4114028.0                VHV   42109  Wuppertal   
1           1  4114039.0                VHV   42277  Wuppertal   
2           2  4114045.0                VHV   42389  Wuppertal   
3           3  4114049.0                VGV   42277  Wuppertal   
4           4  4114055.0                VHV   42553    Velbert   

                STRASSE  SUM_INSURED CONSTRACTION_DESIGN  CONSTRUCTION_YEAR  \
0     Hans-Böckler-Str.      71081.0      NORMAL_VENTURE        1967.565648   
1       Liegnitzer Str.      55708.0      NORMAL_VENTURE        1967.565648   
2             Rascheweg      74148.0      DESIGN_CLASS_I        1967.565648   
3               Am Diek     664000.0      NORMAL_VENTURE        1967.565648   
4  Emil-Schniewind-Str.      75682.0      NORMAL_VENTURE        1967.565648   

         WFL  ... PRIOR_DAMAGES  UVV-KZ  UNDERWRITER              PARTY-ID  \
0   69.00000  ...             0       1           

In [25]:
count_of_Specific = (contract_class['ANO_SID'] == 33145282.0).sum()

# Display the count
print(f"The number appears {count_of_Specific} times.")

# Get the value counts for the 'feature_name' column
value_counts = contract_class['ANO_SID'].value_counts()

# Filter the counts to find entries that appear more than 11 times
entries_more_than_11 = value_counts[value_counts > 11]

# Count how many unique entries exist more than 11 times
count_entries_more_than_11 = len(entries_more_than_11)

# Display the result
print(f"Number of unique entries that appear more than 11 times: {count_entries_more_than_11}")

print(len(contract_class['ANO_SID']))

print(contract_class['ANO_SID'].value_counts())

print(contract_class['ANO_SID'].nunique())

print(contract_class['DAMAGE'].value_counts())



The number appears 3250 times.
Number of unique entries that appear more than 11 times: 7344
10374540
ANO_SID
33145282.0     3250
43075840.0     1460
45987301.0     1360
43514529.0     1305
49624634.0      896
               ... 
31621351.0        1
58677674.0        1
31617266.0        1
31617021.0        1
145240911.0       1
Name: count, Length: 1513666, dtype: int64
1513666
DAMAGE
0    10203688
1      170852
Name: count, dtype: int64


In [5]:
print(contract_class['UNDERWRITER'].value_counts())

UNDERWRITER
Y    10367593
N        6947
Name: count, dtype: int64


In [9]:
# Group by 'underwriter' and count unique 'ANO_SID' entries
unique_ano_sid_counts = contract_class.groupby('UNDERWRITER')['ANO_SID'].nunique()

print("Unique ANO_SID entries for each underwriter:")
print(unique_ano_sid_counts)

print(contract_class.groupby('UNDERWRITER')['ANO_SID'].value_counts())

Unique ANO_SID entries for each underwriter:
UNDERWRITER
N         66
Y    1513600
Name: ANO_SID, dtype: int64
UNDERWRITER  ANO_SID    
N            33145282.0     3250
             45987301.0     1360
             49624634.0      896
             50161558.0      343
             28138288.0      169
                            ... 
Y            46816926.0        1
             142189756.0       1
             46816593.0        1
             46816586.0        1
             145240911.0       1
Name: count, Length: 1513666, dtype: int64


## Load and control original damage data

In [6]:
# Load the damage.csv file
damage_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/damages.csv'
damage_class = pd.read_csv(damage_path)



# Display the first few rows to verify content
print(damage_class.head())

  ABTEILG_LVL01_SEGMENT ABTEILG_LVL02_VERANTWORTUNG ABTEILG_LVL03_KD_SEGMENT  \
0                    KP                      KP-SHU                  KP-SHU1   
1                    KP                      KP-SHU                  KP-SHU1   
2                    KP                      KP-SHU                  KP-SHU1   
3                    KP                      KP-SHU                  KP-SHU1   
4                    KP                      KP-SHU                  KP-SHU1   

   ADDRESS_TYPE  AGT_SCHADENMELDER   EXPENSE     DAMAGE_CLUSTER_ID  \
0           NaN             1487.0   6132.09  000001349d9612449000   
1           NaN             1721.0  11625.05  000001349db8deb42000   
2  HOME_ADDRESS                NaN    148.39  000001349dd338a45000   
3           NaN                NaN    429.29                   NaN   
4           NaN                NaN    366.82                   NaN   

  DAT_ANL_ERSTRESERVIERUNG         DAT_EREIGNIS DAT_ERLFDAT_ERLEDIGUNG_FORD  \
0      2012-01-03 0

<ipython-input-6-c60d1b5c3da2>:3: DtypeWarning: Columns (5,12,18,24,41) have mixed types. Specify dtype option on import or set low_memory=False.
  damage_class = pd.read_csv(damage_path)


In [16]:
count_of_Specific = (damage_class['ANO_SID'] == 33145282).sum()

# Display the count
print(f"The number appears {count_of_Specific} times.")

# Get the value counts for the 'feature_name' column
value_counts = damage_class['ANO_SID'].value_counts()

# Filter the counts to find entries that appear more than 11 times
entries_more_than_11 = value_counts[value_counts > 11]

# Count how many unique entries exist more than 11 times
count_entries_more_than_11 = len(entries_more_than_11)

# Display the result
print(f"Number of unique entries that appear more than 11 times: {count_entries_more_than_11}")

print(len(damage_class['ANO_SID']))

print(damage_class['ANO_SID'].value_counts())

print(damage_class['ANO_SID'].nunique())

The number appears 5042 times.
Number of unique entries that appear more than 11 times: 1552
425738
ANO_SID
33192106    21456
33145282     5042
17675138     1931
45987301     1604
56575465     1579
            ...  
50942493        1
31881813        1
46437021        1
52727978        1
19935049        1
Name: count, Length: 229258, dtype: int64
229258


## Load and controll original contract data

In [21]:
# Load the Contract_14.csv file
contract_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/combined_contracts.csv'
contract = pd.read_csv(contract_path)

<ipython-input-21-4cd68fd886c8>:3: DtypeWarning: Columns (2,5,11,27) have mixed types. Specify dtype option on import or set low_memory=False.
  contract = pd.read_csv(contract_path)


In [23]:
# Get the value counts for the 'feature_name' column
value_counts = contract['ANO_SID'].value_counts()

# Filter the counts to find entries that appear more than 11 times
entries_more_than_11 = value_counts[value_counts > 11]

# Count how many unique entries exist more than 11 times
count_entries_more_than_11 = len(entries_more_than_11)

# Display the result
print(f"Number of unique entries that appear more than 11 times: {count_entries_more_than_11}")

print(len(contract['ANO_SID']))

print(contract['ANO_SID'].value_counts())

print(contract['ANO_SID'].nunique())

# Filter the DataFrame
filtered_df = clean_2[clean_2['ANO_SID'] == 34281783.0]

print("Entries where ANO_SID is 34281783.0:")
print(filtered_df)

Number of unique entries that appear more than 11 times: 478
11534325
ANO_SID
34281783.0     66
31591521.0     66
43075840.0     55
44182579.0     49
32128575.0     44
               ..
58735892.0      1
51504741.0      1
40649149.0      1
40652420.0      1
145240911.0     1
Name: count, Length: 1683447, dtype: int64
1683447
Entries where ANO_SID is 34281783.0:
         Unnamed: 0     ANO_SID CORPORATE_DEVISION  ORTPLZ ORTS-NAME  \
365214       365214  34281783.0                VGV   86152  Augsburg   
365215       365215  34281783.0                VGV   86153  Augsburg   
1290226     1290226  34281783.0                VGV   86152  Augsburg   
1290227     1290227  34281783.0                VGV   86153  Augsburg   
2211582     2211582  34281783.0                VGV   86152  Augsburg   
2211583     2211583  34281783.0                VGV   86153  Augsburg   
3136250     3136250  34281783.0                VGV   86152  Augsburg   
3136251     3136251  34281783.0                VGV   86153  

We can conclude that the outliers have been in the original dataset for both damage and contracts. They do not come from our data merging.

## Cleaning the data

In [13]:
# Option one: Delete entries were UNDERWRITER is N

clean_1 = contract_class[contract_class['UNDERWRITER'] !='N']

# Get the value counts for the 'feature_name' column
value_counts = clean_1['ANO_SID'].value_counts()

# Filter the counts to find entries that appear more than 11 times
entries_more_than_11 = value_counts[value_counts > 11]

# Count how many unique entries exist more than 11 times
count_entries_more_than_11 = len(entries_more_than_11)

# Display the result
print(f"Number of unique entries that appear more than 11 times: {count_entries_more_than_11}")

print(len(clean_1['ANO_SID']))

print(clean_1['ANO_SID'].value_counts())

print(clean_1['ANO_SID'].nunique())



Number of unique entries that appear more than 11 times: 7317
10367593
ANO_SID
43075840.0     1460
43514529.0     1305
43082629.0      752
31110360.0      405
43541292.0      228
               ... 
5485284.0         1
51910474.0        1
5481368.0         1
5481336.0         1
145240911.0       1
Name: count, Length: 1513600, dtype: int64
1513600


=>That does not solve the problem

In [28]:
#Option 2: When a ANO_SID has more than 2 entries i a year, the entries that are too much get deleted. Why 2?

# Group by YEAR and ANO_SID, and then count the entries
grouped = contract_class.groupby(['YEAR', 'ANO_SID']).cumcount()

# Filter out entries where the count is greater than or equal to 2
clean_2 = contract_class[grouped < 2]

# Get the value counts for the 'feature_name' column
value_counts = clean_2['ANO_SID'].value_counts()

# Filter the counts to find entries that appear more than 11 times
entries_more_than_11 = value_counts[value_counts > 11]

# Count how many unique entries exist more than 11 times
count_entries_more_than_11 = len(entries_more_than_11)

# Display the result
print(f"Number of unique entries that appear more than 11 times: {count_entries_more_than_11}")

print(len(clean_2['ANO_SID']))

print(clean_2['ANO_SID'].value_counts())

print(clean_2['ANO_SID'].nunique())

# Filter the DataFrame
filtered_df = clean_2[clean_2['ANO_SID'] == 49357394.0]

print(clean_2['DAMAGE'].value_counts())


Number of unique entries that appear more than 11 times: 7034
10353244
ANO_SID
49357394.0     22
38272577.0     22
5291031.0      22
31591521.0     22
9194454.0      22
               ..
130373237.0     1
53443418.0      1
53443451.0      1
41031570.0      1
145240911.0     1
Name: count, Length: 1513666, dtype: int64
1513666
DAMAGE
0    10203172
1      150072
Name: count, dtype: int64


We could also change to 1 ANO_SID per year

In [ ]:
# Save the DataFrame to a CSV file
clean_2.to_csv('clean_2.csv', index=False)